# Request: Make a get call to:https://api.close.com/buildwithus/
Follow instructions in response body, retrieve and respond to hiring manager with verification ID

In [1]:
import requests
import json
import hashlib

API_URL = "https://api.close.com/buildwithus/"

In [2]:
def safe_json(response):
    """Try to parse JSON, fallback to raw text if not valid JSON."""
    try:
        return response.json()
    except ValueError:
        return {"raw_response": response.text.strip()}

In [3]:
def fetch_traits():
    """GET the traits and key from the API."""
    response = requests.get(API_URL, timeout=10)
    response.raise_for_status()
    return response.json()

In [4]:
def generate_post_body(traits, key, digest_size=64):
    """Generate BLAKE2b keyed hashes for each trait."""
    hashed_traits = []
    key_bytes = key.encode("utf-8")
    
    for trait in traits:
        trait_bytes = trait.encode("utf-8")
        h = hashlib.blake2b(trait_bytes, key=key_bytes, digest_size=digest_size)
        hashed_traits.append(h.hexdigest())
    
    return hashed_traits

In [5]:
def submit_hashes(post_body):
    """POST the hashed array back to the API and safely parse response."""
    headers = {"Content-Type": "application/json"}
    response = requests.post(API_URL, data=json.dumps(post_body), headers=headers, timeout=10)
    
    print(f"[DEBUG] POST status: {response.status_code}")
    print(f"[DEBUG] POST raw: {response.text.strip()[:200]}...")
    
    response.raise_for_status()
    return safe_json(response)


In [6]:
def main():
    # Step 1: GET traits and key
    data = fetch_traits()
    traits = data["traits"]
    key = data["key"]
    
    print("Traits:", traits)
    print("Key:", key)
    
    # Step 2: Generate hashes
    post_body = generate_post_body(traits, key)
    print("Generated hashes:", post_body)
    
    # Step 3: POST hashes and get response
    result = submit_hashes(post_body)
    print("API Response:", result)

In [7]:
if __name__ == "__main__":
    main()

Traits: ['Craftsman', 'Pragmatic', 'Curious', 'Methodical', 'Driven', 'Collaborator']
Key: Close-3b55a485
Generated hashes: ['5c37eab6391a9260d613962500aa53c87e2716b30d29333784577b9e583a6e9ec6549428ba9cd6f8773c24e7e00ef404b370624509a849115222ad40aae4e9f6', 'ea4e0714975462add996d9003228739a6d43a7be5b79bd0b57fca853f7531717ac77416144491e351508d16132cde91f0255d40cca9296024892a4280523925a', 'a31f8c3e7f9e5c82b62f9a6d98a172132fee9bfa6beef3d11a3f975a4f952981c0d321abab5a79f4f270cc757b9c45a8f534fffc608362803241c578a405def9', '785c87d191c31fcdc1c4bd827ef5d0e512c29ab352ac8bfd4095a4ab77815151340a9f70463f1d68d709494f8ebd2f33212f7d4af9a2fc26d9d563180d526f23', 'b61b7d4ebf29b078cecf2ca064e372fc7ffa662c338fcf25351e6adc5e1f9e3cf55ce25695fe5f6cb9eed3834438e87d87e90954cf7448966f381d610e637a7c', '14c21e7fde639ad0e2bdf44437cf3325e2b729ae5619f0417358604cccfddc75e6f8a3fbda939e154551d249edf824b2cb63cdeccadf2c24571744f8e794767f']
[DEBUG] POST status: 200
[DEBUG] POST raw: Verification ID: 1753650208.ah1FzID-M24.